In [15]:
import numpy as np
import pandas as pd

In [16]:
df = pd.read_csv('desafio-base1/agenda.csv', parse_dates=['Data'], encoding='iso-8859-1', quotechar='"', delimiter='|')
df.shape

(1230, 9)

In [17]:
df.head(3).T

,0,1,2
Código,91,100,564
Data,2020-09-10 00:00:00,1997-06-01 00:00:00,1999-09-10 00:00:00
CodConvenio,9.0,13.0,2.0
HoraConsulta,14:13:35,NaN,18:31:24
Tipo,1ª vez,Consulta,Retorno
Valor,107.0,123.0,212.0
HoraAtendimento,19:00:49,NaN,22:31:27
HoraFim,22:15:06,NaN,23:11:29
Usuário,José,Teste 1,Teste 1


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Código           1230 non-null   int64         
 1   Data             1230 non-null   datetime64[ns]
 2   CodConvenio      986 non-null    float64       
 3   HoraConsulta     984 non-null    object        
 4   Tipo             995 non-null    object        
 5   Valor            977 non-null    float64       
 6   HoraAtendimento  984 non-null    object        
 7   HoraFim          984 non-null    object        
 8   Usuário          1230 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 86.6+ KB


In [19]:
# Trantando recursos "HoraConsulta", "HoraAtendimento" e "HoraFim"
# Como não foi informado a data de entrada, deixaremos só o tempo na coluna de arrival_time
df['HoraConsulta'] = pd.to_datetime(df['HoraConsulta'], format='%H:%M:%S').dt.time
df['HoraAtendimento'] = pd.to_datetime(df['HoraAtendimento'], format='%H:%M:%S').dt.time
df['HoraFim'] = pd.to_datetime(df['HoraFim'], format='%H:%M:%S').dt.time

In [20]:
# Tratamento recurso 'patient_name'
# Este recurso precisa de informação externa para ser tratado, resumidamente, utilizaremos o índice em "patient_id" para localizar o nome do usuário no arquivo "patient.csv"
df_patient = pd.read_csv('patient.csv', usecols = ["name"])

In [21]:
# Neste caso, como o 'patient_id' inicia com 0 e está ordenada é fácil de encontrar seu nome correspondente
index = df['Código'].values
df['patient_name'] = df_patient['name'][index].values

In [22]:
# Tratamento recurso "physician_id"
# Será criado um novo recurso com o nome "physician_id", como a seguir:
df['physician_id'] = pd.factorize(df['Usuário'])[0].astype(int)


In [33]:
# Exportando os nomes dos médicos
uniques = pd.factorize(df['Usuário'])[1]
pd_uniques = pd.DataFrame(data={"physician_names": uniques})
pd_uniques.to_csv('physician_names.csv',index=False, encoding='utf-8')


In [9]:
# Tratamento recurso 'eventprocedure_pack'
# Será criado um novo recurso, foi utilizado force_ascii para visualizar a escrita de forma correta em utf-8 (não temos a informação 'quantity')
df['eventprocedure_pack'] = ('json::['+df[['Tipo','Valor']].apply(lambda x: x.to_json(force_ascii =  False),axis=1)+']')

In [10]:
# Colocando o nome das colunas do arquivo de acordo a documentação da iClinic
df = df.rename(
    columns = {
        "Código": "patient_id",
        "Data": "date",
        "HoraConsulta": "arrival_time",
        "HoraAtendimento": "start_time",
        "HoraFim": "end_time",
        "CodConvenio": "healthinsurance_name",
        "Tipo": "name",
        "Valor": "value",
    }
)

In [11]:
#Adiconando as colunas ausentes
df['status'] = 'cp'
df['patient_home_phone'] = np.nan
df['patient_mobile_phone'] = np.nan
df['description'] = np.nan
df['all_day'] = np.nan
df['cancel_reason'] = np.nan  
df['patient_email'] = np.nan  
df['event_blocked_scheduling'] = np.nan
df['quantity'] = np.nan  

In [12]:
df = df.loc[:, ["patient_id","patient_name","physician_id","date","status","patient_home_phone","patient_mobile_phone","arrival_time","start_time","end_time","description","all_day","cancel_reason","patient_email","event_blocked_scheduling","healthinsurance_name","eventprocedure_pack"]]

In [13]:
df.to_csv('event_scheduling.csv',index=False, encoding='utf-8')
